In [1]:
import logging

import numpy as np
import matplotlib.pyplot as plt

try:
    import keysightSD1
except:
    import sys
    sys.path.append(r"C:\Program Files (x86)\Keysight\SD1\Libraries\Python")
    import keysightSD1
    
import qcodes
from qcodes_contrib_drivers.drivers.Keysight.M3202A import M3202A

import qcodes.logger as logger
from qcodes.logger import start_all_logging

start_all_logging()
# logger.get_file_handler().setLevel(logging.DEBUG)

ModuleNotFoundError: No module named 'qcodes_contrib_drivers'

In [ ]:
# try to close station from previous run.

try:
    station.close_all_registered_instruments()
except: pass

In [ ]:
# setup station
awg = M3202A("AWG", chassis = 0, slot = 4)

station = qcodes.Station()
station.add_component(awg)

In [ ]:
# setup output channels
pxi = keysightSD1.SD_TriggerExternalSources.TRIGGER_PXI1
trigger_mode = keysightSD1.SD_TriggerBehaviors.TRIGGER_FALL

for ch in range(1,5):
    awg.set_channel_offset(0.5, ch)
    awg.set_channel_amplitude(0.5, ch)

    awg.set_channel_wave_shape(keysightSD1.SD_Waveshapes.AOU_AWG, ch)
    awg.awg_queue_config(ch, keysightSD1.SD_QueueMode.CYCLIC)
    awg.awg_config_external_trigger(ch, pxi, trigger_mode)


In [ ]:
# setup waveforms
def get_divider(prescaler):
    if prescaler == 0:
        divider = 1
    elif prescaler == 1:
        divider = 5
    else:
        divider = prescaler * 10
        
    return divider

def create_sine(period, repetition, prescaler):
    divider = get_divider(prescaler)
    n_pts = repetition * period // divider
    phi = np.linspace(0, np.pi*2*repetition, n_pts)
    w = np.sin(phi)
    
    if len(w) < 2000:
        raise Exception("not enough data")
        
    return w

prescaler_1GSa = 0
sine = 0.6 * create_sine(500, 4, prescaler_1GSa)
sine_awg = awg.upload_waveform(sine)

delay = 0
ext_trigger = keysightSD1.SD_TriggerModes.EXTTRIG
auto_trigger = keysightSD1.SD_TriggerModes.AUTOTRIG
cycles = 1
awg.awg_queue_waveform(1, sine_awg, ext_trigger, delay, cycles, prescaler_1GSa)

awg.awg_start(0)

In [ ]:
# trigger AWG

awg.set_pxi_trigger(0, pxi1) # trigger ON
awg.set_pxi_trigger(1, pxi1) # trigger OFF